In [17]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection

def find_counts(predictions, truth):
    TP = 0; TN = 0; FP = 0; FN = 0
    for i in range(predictions.shape[0]):
        if predictions[i] < 0.5 and truth[i] == 0:
            TN += 1
        elif predictions[i] >= 0.5 and truth[i] == 1:
            TP += 1
        elif predictions[i] < 0.5 and truth[i] == 1:
            FN += 1
        elif predictions[i] >= 0.5 and truth[i] == 0:
            FP += 1
    return TP, TN, FP, FN

def find_accuracies(TP, TN, FP, FN):
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    a_0 = TP / (TP + FN)
    a_1 = TN / (TN + FP)
    balanced_accuracy = 1/2 * (a_0 + a_1)
    return accuracy, balanced_accuracy, a_0, a_1

def main():
    X = np.load("../util/X_mfccs/X_mfccs_4000.npy")
    X = np.reshape(X, (X.shape[0], -1))
    y = np.load("../util/labels/labels_4000.npy")
    x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.10, random_state=42)

    theta_0 = np.zeros((x_train.shape[1],))
    clf = LogisticRegression(theta_0 = theta_0, verbose = True)
    clf.fit(x_train, y_train)
    
    predictions = clf.predict(x_valid)
    TP, TN, FP, FN = find_counts(predictions, y_valid)
    print(find_accuracies(TP, TN, FP, FN))


class LogisticRegression:
    """Logistic regression with Newton's Method as the solver.

    Example usage:
        > clf = LogisticRegression()
        > clf.fit(x_train, y_train)
        > clf.predict(x_eval)
    """
    def __init__(self, step_size=0.01, max_iter=10, eps=1e-5,
                 theta_0=None, verbose=True):
        """
        Args:
            step_size: Step size for iterative solvers only.
            max_iter: Maximum number of iterations for the solver.
            eps: Threshold for determining convergence.
            theta_0: Initial guess for theta. If None, use the zero vector.
            verbose: Print loss values during training.
        """
        self.theta = theta_0
        self.step_size = step_size
        self.max_iter = max_iter
        self.eps = eps
        self.verbose = verbose

    def fit(self, x, y):
        """Run Newton's Method to minimize J(theta) for logistic regression.

        Args:
            x: Training example inputs. Shape (n_examples, dim).
            y: Training example labels. Shape (n_examples,).
        """
        for _ in range(self.max_iter):
            g = gradient(x, y, self.theta)
            print("calculated gradient")
            h = hessian(x, y, self.theta)
            print("calculated hessian")
            old_theta = self.theta
            self.theta = old_theta - self.step_size * np.dot(np.linalg.inv(h), g)
            if self.verbose:
                print(loss(x, y, self.theta))
            if np.linalg.norm(self.theta - old_theta, ord = 1) < self.eps:
                break
            

    def predict(self, x):
        """Return predicted probabilities given new inputs x.

        Args:
            x: Inputs of shape (n_examples, dim).

        Returns:
            Outputs of shape (n_examples,).
        """
        return sigmoid(np.dot(x, self.theta))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def gradient(x, y, theta):
    XT_theta = np.dot(x, theta)
    sigmoid_XT_theta = sigmoid(XT_theta)
    S = y - sigmoid_XT_theta
    return (-1/x.shape[0]) * np.dot(x.T, S)

def loss(x, y, theta):
    loss = 0
    for i in range(y.shape[0]):
        if (y[i]) == 1:
            loss += y[i]*np.log(sigmoid(np.dot(theta.T, x[i, :])))
        if (y[i]) == 0:
            loss += (1-y[i])*np.log(1-sigmoid(np.dot(theta.T, x[i, :])))
    return (-1/y.shape[0] * loss)

def hessian(x, y, theta):
    XT_theta = np.dot(x, theta)
    sigmoid_XT_theta = sigmoid(XT_theta)
    diag = sigmoid_XT_theta * (1 - sigmoid_XT_theta)
    G = np.diag(diag)
    return (1/x.shape[0]) * np.dot(np.dot(x.T, G), x)

if __name__ == '__main__':
    main()


calculated gradient
calculated hessian
0.6882676344714105
calculated gradient
calculated hessian
0.6834847444384837
calculated gradient
calculated hessian
0.6787957370349714
calculated gradient
calculated hessian
0.6741979522926653
calculated gradient
calculated hessian
0.669688837678916
calculated gradient
calculated hessian
0.6652659424638303
calculated gradient
calculated hessian
0.6609269124477684
calculated gradient
calculated hessian
0.6566694850215847
calculated gradient
calculated hessian
0.6524914845352293
calculated gradient
calculated hessian
0.6483908179517811
(0.8225520602569782, 0.8046353986749919, 0.7518443997317237, 0.85742639761826)
